In [1]:
'''
  code by Tae Hwan Jung(Jeff Jung) @graykode
'''
import numpy as np
import torch
#神經網路
import torch.nn as nn
#各種優化算法
import torch.optim as optim
#自動求導數
from torch.autograd import Variable
import torch.nn.functional as F

dtype = torch.FloatTensor

# Text-CNN Parameter
embedding_size = 2 # 詞向量的維度
sequence_length = 3
num_classes = 2  # 0 or 1
filter_sizes = [2, 2, 2] # n-gram window
num_filters = 3

# 3 words sentences (=sequence_length is 3)
sentences = ["i love you", "he loves me", "she likes baseball", "i hate you", "sorry for that", "this is awful"]
labels = [1, 1, 1, 0, 0, 0]  # 1 is good, 0 is not good.

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}
vocab_size = len(word_dict)
print(vocab_size)

inputs = []
for sen in sentences:
    inputs.append(np.asarray([word_dict[n] for n in sen.split()]))
print(len(inputs))
targets = []
for out in labels:
    targets.append(out) # To using Torch Softmax Loss function

input_batch = Variable(torch.LongTensor(inputs))
print(input_batch.shape)
target_batch = Variable(torch.LongTensor(targets))
print(target_batch)
class TextCNN(nn.Module):
    def __init__(self):
        super(TextCNN, self).__init__()
        # 總共會有num_filters * len(filter_sizes)個卷積核的輸出
        self.num_filters_total = num_filters * len(filter_sizes)
        # W為我們的嵌入詞典，第一維為詞典的大小，第二維為每個詞的詞向量的嵌入維度。其中每一行為一個詞的詞向量。
        self.W = nn.Parameter(torch.empty(vocab_size, embedding_size).uniform_(-1, 1)).type(dtype)
        print(self.W)
        # 最後輸出層的權重，維度為(num_filters_total, num_classes)。
        self.Weight = nn.Parameter(torch.empty(self.num_filters_total, num_classes).uniform_(-1, 1)).type(dtype)
        # 最後輸出層的bias
        self.Bias = nn.Parameter(0.1 * torch.ones([num_classes])).type(dtype)

    def forward(self, X):
        # 將輸入轉化為詞向量，維度為[batch_size, sequence_length, sequence_length]
        embedded_chars = self.W[X] 
        print(embedded_chars)
        # 添加額外的一個channel(=1)，為了適應Conv2d。維度為[batch, channel(=1), sequence_length, embedding_size]
        embedded_chars = embedded_chars.unsqueeze(1) 
        pooled_outputs = []
        for filter_size in filter_sizes:
            # conv : [input_channel(=1), output_channel(=3), (filter_height, filter_width), bias_option]
            conv = nn.Conv2d(1, num_filters, (filter_size, embedding_size), bias=True)(embedded_chars)
            h = F.relu(conv)
            # mp : ((filter_height, filter_width))
            mp = nn.MaxPool2d((sequence_length - filter_size + 1, 1))
            # pooled : [batch_size(=6), output_height(=1), output_width(=1), output_channel(=3)]
            pooled = mp(h).permute(0, 3, 2, 1)
            pooled_outputs.append(pooled)

        h_pool = torch.cat(pooled_outputs, len(filter_sizes)) # [batch_size(=6), output_height(=1), output_width(=1), output_channel(=3) * 3]
        h_pool_flat = torch.reshape(h_pool, [-1, self.num_filters_total]) # [batch_size(=6), output_height * output_width * (output_channel * 3)]

        model = torch.mm(h_pool_flat, self.Weight) + self.Bias # [batch_size, num_classes]
        return model

model = TextCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training
for epoch in range(5000):
    optimizer.zero_grad()
    output = model(input_batch)

    # output : [batch_size, num_classes], target_batch : [batch_size] (LongTensor, not one-hot)
    loss = criterion(output, target_batch)
    
    if (epoch + 1) % 1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()

# Test
test_text = 'sorry hate you'
tests = [np.asarray([word_dict[n] for n in test_text.split()])]
test_batch = Variable(torch.LongTensor(tests))

# Predict
predict = model(test_batch).data.max(1, keepdim=True)[1]
if predict[0][0] == 0:
    print(test_text,"is Bad Mean...")
else:
    print(test_text,"is Good Mean!!")

16
6
torch.Size([6, 3])
tensor([1, 1, 1, 0, 0, 0])
Parameter containing:
tensor([[-0.3962, -0.9944],
        [ 0.3783, -0.0988],
        [-0.8973,  0.3972],
        [-0.0756,  0.2531],
        [-0.9290, -0.5933],
        [-0.7448, -0.9448],
        [ 0.9964,  0.6607],
        [ 0.7130,  0.7256],
        [ 0.7278,  0.8840],
        [-0.1993, -0.5791],
        [ 0.6343, -0.8486],
        [ 0.4235, -0.7311],
        [ 0.8845,  0.8907],
        [ 0.3052, -0.4557],
        [ 0.3112, -0.4683],
        [ 0.1069,  0.7204]], requires_grad=True)
tensor([[[-0.7448, -0.9448],
         [ 0.7130,  0.7256],
         [-0.3962, -0.9944]],

        [[ 0.1069,  0.7204],
         [ 0.9964,  0.6607],
         [ 0.3783, -0.0988]],

        [[ 0.4235, -0.7311],
         [-0.1993, -0.5791],
         [-0.9290, -0.5933]],

        [[-0.7448, -0.9448],
         [ 0.3112, -0.4683],
         [-0.3962, -0.9944]],

        [[ 0.6343, -0.8486],
         [ 0.8845,  0.8907],
         [-0.0756,  0.2531]],

        [[-0.

tensor([[[-0.7348, -0.9321],
         [ 0.6775,  0.7033],
         [-0.3687, -0.9715]],

        [[ 0.1350,  0.7027],
         [ 0.9553,  0.6349],
         [ 0.3902, -0.0791]],

        [[ 0.3964, -0.7108],
         [-0.1976, -0.5646],
         [-0.8861, -0.5843]],

        [[-0.7348, -0.9321],
         [ 0.3359, -0.4744],
         [-0.3687, -0.9715]],

        [[ 0.6255, -0.8711],
         [ 0.9064,  0.9105],
         [-0.1076,  0.2528]],

        [[-0.9203,  0.4069],
         [ 0.3107, -0.4792],
         [ 0.7326,  0.9016]]], grad_fn=<IndexBackward>)
tensor([[[-0.7346, -0.9314],
         [ 0.6772,  0.7033],
         [-0.3686, -0.9715]],

        [[ 0.1351,  0.7027],
         [ 0.9551,  0.6346],
         [ 0.3900, -0.0792]],

        [[ 0.3960, -0.7103],
         [-0.1974, -0.5641],
         [-0.8855, -0.5843]],

        [[-0.7346, -0.9314],
         [ 0.3358, -0.4747],
         [-0.3686, -0.9715]],

        [[ 0.6258, -0.8714],
         [ 0.9064,  0.9105],
         [-0.1079,  0.2529]

tensor([[[-0.7194, -0.9151],
         [ 0.6681,  0.6837],
         [-0.3598, -0.9576]],

        [[ 0.1433,  0.6848],
         [ 0.9268,  0.6178],
         [ 0.3940, -0.0708]],

        [[ 0.3818, -0.6904],
         [-0.1879, -0.5498],
         [-0.8572, -0.5701]],

        [[-0.7194, -0.9151],
         [ 0.3427, -0.4722],
         [-0.3598, -0.9576]],

        [[ 0.6027, -0.8812],
         [ 0.9196,  0.9096],
         [-0.1174,  0.2561]],

        [[-0.9349,  0.4002],
         [ 0.3049, -0.4829],
         [ 0.7352,  0.9152]]], grad_fn=<IndexBackward>)
tensor([[[-0.7193, -0.9149],
         [ 0.6681,  0.6833],
         [-0.3598, -0.9575]],

        [[ 0.1434,  0.6847],
         [ 0.9264,  0.6176],
         [ 0.3937, -0.0708]],

        [[ 0.3816, -0.6903],
         [-0.1878, -0.5495],
         [-0.8569, -0.5703]],

        [[-0.7193, -0.9149],
         [ 0.3429, -0.4723],
         [-0.3598, -0.9575]],

        [[ 0.6024, -0.8814],
         [ 0.9201,  0.9097],
         [-0.1174,  0.2564]

tensor([[[-0.7188, -0.9037],
         [ 0.6517,  0.6687],
         [-0.3552, -0.9353]],

        [[ 0.1388,  0.6773],
         [ 0.8929,  0.6061],
         [ 0.3929, -0.0692]],

        [[ 0.3569, -0.6666],
         [-0.1875, -0.5336],
         [-0.8446, -0.5670]],

        [[-0.7188, -0.9037],
         [ 0.3767, -0.4798],
         [-0.3552, -0.9353]],

        [[ 0.6128, -0.9027],
         [ 0.9378,  0.9181],
         [-0.1194,  0.2540]],

        [[-0.9387,  0.3927],
         [ 0.3246, -0.5009],
         [ 0.7416,  0.9186]]], grad_fn=<IndexBackward>)
tensor([[[-0.7187, -0.9037],
         [ 0.6516,  0.6685],
         [-0.3554, -0.9350]],

        [[ 0.1386,  0.6769],
         [ 0.8927,  0.6061],
         [ 0.3926, -0.0690]],

        [[ 0.3567, -0.6666],
         [-0.1874, -0.5335],
         [-0.8444, -0.5670]],

        [[-0.7187, -0.9037],
         [ 0.3767, -0.4796],
         [-0.3554, -0.9350]],

        [[ 0.6130, -0.9027],
         [ 0.9378,  0.9182],
         [-0.1193,  0.2537]

tensor([[[-0.6991, -0.9035],
         [ 0.6398,  0.6454],
         [-0.3537, -0.9296]],

        [[ 0.1390,  0.6614],
         [ 0.8794,  0.5791],
         [ 0.3807, -0.0584]],

        [[ 0.3496, -0.6617],
         [-0.1811, -0.5268],
         [-0.8240, -0.5588]],

        [[-0.6991, -0.9035],
         [ 0.3759, -0.4708],
         [-0.3537, -0.9296]],

        [[ 0.6220, -0.9105],
         [ 0.9348,  0.9274],
         [-0.1286,  0.2525]],

        [[-0.9528,  0.3908],
         [ 0.3247, -0.5032],
         [ 0.7583,  0.9256]]], grad_fn=<IndexBackward>)
tensor([[[-0.6987, -0.9030],
         [ 0.6393,  0.6451],
         [-0.3533, -0.9292]],

        [[ 0.1394,  0.6612],
         [ 0.8791,  0.5787],
         [ 0.3805, -0.0583]],

        [[ 0.3494, -0.6616],
         [-0.1812, -0.5267],
         [-0.8238, -0.5586]],

        [[-0.6987, -0.9030],
         [ 0.3761, -0.4705],
         [-0.3533, -0.9292]],

        [[ 0.6217, -0.9105],
         [ 0.9351,  0.9278],
         [-0.1288,  0.2523]

...

tensor([[[-0.2639,  0.0432],
         [-0.2633,  0.0431],
         [-0.2639,  0.0433]],

        [[ 0.0247,  0.0102],
         [ 0.0255,  0.0097],
         [ 0.0251,  0.0091]],

        [[-0.0119, -0.0385],
         [-0.0124, -0.0383],
         [-0.0125, -0.0397]],

        [[-0.2639,  0.0432],
         [ 3.3355, -1.2225],
         [-0.2639,  0.0433]],

        [[ 0.4439, -2.5216],
         [ 2.0476,  2.8846],
         [-0.9491, -0.3265]],

        [[-2.4092,  1.5775],
         [ 1.1345, -2.3103],
         [ 1.1225,  2.5309]]], grad_fn=<IndexBackward>)
tensor([[[-0.2640,  0.0433],
         [-0.2634,  0.0434],
         [-0.2639,  0.0429]],

        [[ 0.0247,  0.0102],
         [ 0.0254,  0.0097],
         [ 0.0251,  0.0091]],

        [[-0.0119, -0.0384],
         [-0.0123, -0.0383],
         [-0.0125, -0.0397]],

        [[-0.2640,  0.0433],
         [ 3.3362, -1.2226],
         [-0.2639,  0.0429]],

        [[ 0.4441, -2.5219],
         [ 2.0479,  2.8848],
         [-0.9492, -0.3266]

tensor([[[-0.2664,  0.0449],
         [-0.2667,  0.0447],
         [-0.2670,  0.0450]],

        [[ 0.0193,  0.0055],
         [ 0.0169,  0.0068],
         [ 0.0180,  0.0071]],

        [[-0.0185, -0.0393],
         [-0.0176, -0.0402],
         [-0.0176, -0.0405]],

        [[-0.2664,  0.0449],
         [ 3.3650, -1.2312],
         [-0.2670,  0.0450]],

        [[ 0.4415, -2.5342],
         [ 2.0611,  2.9126],
         [-0.9625, -0.3361]],

        [[-2.4192,  1.5912],
         [ 1.1455, -2.3277],
         [ 1.1113,  2.5417]]], grad_fn=<IndexBackward>)
tensor([[[-0.2659,  0.0451],
         [-0.2665,  0.0447],
         [-0.2669,  0.0450]],

        [[ 0.0198,  0.0058],
         [ 0.0167,  0.0069],
         [ 0.0181,  0.0069]],

        [[-0.0181, -0.0392],
         [-0.0176, -0.0403],
         [-0.0176, -0.0404]],

        [[-0.2659,  0.0451],
         [ 3.3652, -1.2312],
         [-0.2669,  0.0450]],

        [[ 0.4415, -2.5345],
         [ 2.0611,  2.9129],
         [-0.9625, -0.3362]

tensor([[[-2.6292e-01,  4.3385e-02],
         [-2.6248e-01,  4.4129e-02],
         [-2.6367e-01,  4.5248e-02]],

        [[ 2.0324e-02,  4.0648e-03],
         [ 1.9278e-02,  3.3093e-03],
         [ 2.0026e-02,  3.1285e-03]],

        [[-1.4222e-02, -3.8883e-02],
         [-1.4792e-02, -4.0313e-02],
         [-1.4812e-02, -3.9980e-02]],

        [[-2.6292e-01,  4.3385e-02],
         [ 3.3939e+00, -1.2349e+00],
         [-2.6367e-01,  4.5248e-02]],

        [[ 4.4010e-01, -2.5536e+00],
         [ 2.0706e+00,  2.9292e+00],
         [-9.8018e-01, -3.3654e-01]],

        [[-2.4277e+00,  1.5922e+00],
         [ 1.1557e+00, -2.3411e+00],
         [ 1.1089e+00,  2.5540e+00]]], grad_fn=<IndexBackward>)
tensor([[[-2.6289e-01,  4.3467e-02],
         [-2.6221e-01,  4.4190e-02],
         [-2.6354e-01,  4.5061e-02]],

        [[ 2.0349e-02,  4.0206e-03],
         [ 1.9435e-02,  3.1914e-03],
         [ 2.0078e-02,  3.0816e-03]],

        [[-1.4131e-02, -3.8958e-02],
         [-1.4683e-02, -4.0408e-02

tensor([[[-2.6019e-01,  4.2748e-02],
         [-2.5815e-01,  4.3207e-02],
         [-2.5896e-01,  4.4679e-02]],

        [[ 1.3875e-02,  3.2787e-03],
         [ 1.5563e-02,  4.2923e-03],
         [ 1.5268e-02,  5.9892e-03]],

        [[-1.7835e-02, -3.8532e-02],
         [-1.8396e-02, -3.6813e-02],
         [-1.6711e-02, -3.5720e-02]],

        [[-2.6019e-01,  4.2748e-02],
         [ 3.4080e+00, -1.2424e+00],
         [-2.5896e-01,  4.4679e-02]],

        [[ 4.3461e-01, -2.5701e+00],
         [ 2.0882e+00,  2.9481e+00],
         [-9.8578e-01, -3.4867e-01]],

        [[-2.4312e+00,  1.5959e+00],
         [ 1.1670e+00, -2.3549e+00],
         [ 1.1076e+00,  2.5633e+00]]], grad_fn=<IndexBackward>)
tensor([[[-2.6023e-01,  4.2704e-02],
         [-2.5812e-01,  4.3324e-02],
         [-2.5916e-01,  4.4358e-02]],

        [[ 1.4026e-02,  3.1979e-03],
         [ 1.5337e-02,  4.4410e-03],
         [ 1.5233e-02,  5.9259e-03]],

        [[-1.7880e-02, -3.8404e-02],
         [-1.8376e-02, -3.6795e-02

tensor([[[-0.2590,  0.0463],
         [-0.2596,  0.0458],
         [-0.2608,  0.0452]],

        [[ 0.0128,  0.0043],
         [ 0.0118,  0.0041],
         [ 0.0128,  0.0038]],

        [[-0.0197, -0.0362],
         [-0.0191, -0.0360],
         [-0.0198, -0.0355]],

        [[-0.2590,  0.0463],
         [ 3.4396, -1.2446],
         [-0.2608,  0.0452]],

        [[ 0.4283, -2.5821],
         [ 2.0945,  2.9594],
         [-0.9952, -0.3543]],

        [[-2.4449,  1.6058],
         [ 1.1773, -2.3688],
         [ 1.1047,  2.5865]]], grad_fn=<IndexBackward>)
tensor([[[-0.2589,  0.0465],
         [-0.2595,  0.0460],
         [-0.2609,  0.0451]],

        [[ 0.0126,  0.0045],
         [ 0.0120,  0.0042],
         [ 0.0129,  0.0039]],

        [[-0.0196, -0.0360],
         [-0.0190, -0.0357],
         [-0.0196, -0.0356]],

        [[-0.2589,  0.0465],
         [ 3.4399, -1.2449],
         [-0.2609,  0.0451]],

        [[ 0.4282, -2.5823],
         [ 2.0945,  2.9596],
         [-0.9952, -0.3543]